In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Difference Group By and Reduce By Key") \
        .master("yarn") \
        .getOrCreate()

25/02/11 16:02:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

In [3]:
!hadoop fs -ls -h /ecommerce_data/ecommerce_data/300MB

Found 5 items
-rw-r--r--   2 lokesh hadoop    327.4 M 2025-02-10 16:01 /ecommerce_data/ecommerce_data/300MB/customers.csv
-rw-r--r--   2 lokesh hadoop    275.3 M 2025-02-10 16:01 /ecommerce_data/ecommerce_data/300MB/items.csv
-rw-r--r--   2 lokesh hadoop    271.0 M 2025-02-10 16:01 /ecommerce_data/ecommerce_data/300MB/orders.csv
-rw-r--r--   2 lokesh hadoop    268.4 M 2025-02-10 16:01 /ecommerce_data/ecommerce_data/300MB/payments.csv
-rw-r--r--   2 lokesh hadoop    256.5 M 2025-02-10 16:01 /ecommerce_data/ecommerce_data/300MB/shippings.csv


In [4]:
hdfs_path = "/ecommerce_data/ecommerce_data/300MB/customers.csv"

In [5]:
customer_rdd = spark.sparkContext.textFile(hdfs_path)

In [6]:
header = customer_rdd.first()

In [7]:
customer_rdd = customer_rdd.filter(lambda row : row != header)

In [8]:
customer_rdd = customer_rdd.map(lambda row : row.split(","))

In [9]:
customer_rdd.first()

['0', 'Customer_0', 'Pune', 'Maharashtra', 'India', '2023-01-19', 'True']

In [10]:
city_rdd = customer_rdd.map(lambda row : (row[2], 1))

### **Reduce By**

In [11]:
reduced_rdd = city_rdd.reduceByKey(lambda x, y : x + y)

In [12]:
reduced_rdd.collect()

[('Delhi', 661025),
 ('Chennai', 660249),
 ('Kolkata', 660174),
 ('Bangalore', 661013),
 ('Pune', 660737),
 ('Ahmedabad', 660218),
 ('Mumbai', 661241),
 ('Hyderabad', 662281)]

### **Group By Key**

In [13]:
grouped_rdd = city_rdd.groupByKey()

In [14]:
grouped_result = grouped_rdd.map(lambda x : (x[0],len(x[1])))

In [15]:
grouped_result.collect()

[('Delhi', 661025),
 ('Pune', 660737),
 ('Kolkata', 660174),
 ('Chennai', 660249),
 ('Bangalore', 661013),
 ('Mumbai', 661241),
 ('Ahmedabad', 660218),
 ('Hyderabad', 662281)]

In [16]:
spark.stop()